In [1]:
import sys
sys.path.append('..')

In [2]:
from copy import deepcopy
from random import shuffle, choice
from library.solution import Solution
from library.algorithms.hill_climbing import hill_climbing

In previous notebooks, we defined the `Solution` class as an **abstract class** with methods that must be implemented by subclasses, depending on the specific optimization problem. While the implementation of solutions depend on the problem, all solutions share common principles: they require a **representation**, a **fitness function**, and a method for **random initialization**.

By extending this class, we can define solution classes specific to different optimization problems. For example, we created the `IntBinSolution` class to represent solutions for the IntBin optimization problem.

We then applied the Hill Climbing algorithm to the IntBin problem by further extending `IntBinSolution` to implement the `get_neighbors()` method, which is essential for Hill Climbing algorithm. To do this, we created a new class, `IntBinHillClimbingSolution`.

Today, we'll use Hill Climbing to solve two new problems: the Traveling Salesperson Problem (TSP) and the Knapsack Problem (KS).

## Traveling Salesperson Problem

**Description:** The Traveling Salesperson Problem (TSP) is the challenge of finding the shortest possible route that starts in a given city, visits each of the remaining N-1 cities exactly once, and returns to the starting city.

**Search space:** All possible permutations of city visit orders, forming valid round-trip routes.

**Representation:** List of city indexes that compose the route

**Fitness function:** f(x) = Total distance traveled, computed as the sum of distances between consecutive cities in the route.

**Goal:** Minimize f(x).

Let's begin by implementing the `TSPSolution` class, which inherits from `Solution`. As a result, it must implement the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Traveling Salesperson Problem (TSP) and does not include any implementation related to the optimization algorithm that will be used to solve it.

![TSP Solution](images/tsp-solution.png)

In [ ]:
from library.problems.data.tsp_data import distance_matrix

#TODO: Implement TSPSolution class

class TSPSolution(Solution):
    def __init__(self, data = distance_matrix, repr = None, starting_index = 0):
        self.data = data
        self.starting_index = starting_index
        super().__init__(repr = repr)
        
    def random_initial_representation(self):
        route = [self.starting_index]
        cities = list(range(len(self.data)))
        cities.remove(self.starting_index)
        shuffle(cities)
        route = route + cities
        route = route + [self.starting_index]
        return route
    
    def fitness(self):
        total_distance = 0
        for route_idx in range(0, len(self.repr) - 1):
            current_city = self.repr[route_idx]
            next_city = self.repr[route_idx + 1]
            distance = self.data[current_city][next_city]
            total_distance += distance
            
        return total_distance

Let's test it

In [17]:
solution = TSPSolution(starting_index = 4)

print('Random solution:', solution)
print('Fitness:', solution.fitness())

Random solution: [4, 10, 3, 11, 9, 0, 8, 6, 12, 5, 1, 2, 7, 4]
Fitness: 15916


### Solving TSP with Hill Climbing

To use Hill Climbing to solve TSP we need to define a `TSPHillClimbingSolution` class that implements the `get_neighbors()` method. We also need to ensure that this function returns a list of solutions that also implement the `get_neighbors()` method, therefore, return a list of solutions of type `TSPHillClimbingSolution` too.

A TSP neighbor solution is obtained by swapping the positions of two consecutive cities in the route.

![TSP Hill Climbing Solution](images/tsp-hillclimbing-solution.png)

In [28]:
#TODO: Implement TSPSHillClimbingSolution class
class TSPHillClimbingSolution(TSPSolution):
    def get_neighbors(self):
        neighbors = []
        
        for route in range(1, len(self.repr)-2 ):
            neighbor = deepcopy(self.repr)
            # Swap cities
            neighbor[route], neighbor[route + 1] = neighbor[route + 1], neighbor[route]
            neighbor = TSPHillClimbingSolution(
                repr = neighbor,
                data = self.data,
                starting_index = self.starting_index
            )
            
            neighbors.append(neighbor)
        
        return neighbors
            

Let's test it

In [29]:
solution = TSPHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neighbors:')
for neighbor in neighbors:
    print(neighbor)

Solution: [0, 6, 9, 5, 10, 11, 7, 2, 8, 4, 1, 12, 3, 0]
Neighbors:
[0, 9, 6, 5, 10, 11, 7, 2, 8, 4, 1, 12, 3, 0]
[0, 6, 5, 9, 10, 11, 7, 2, 8, 4, 1, 12, 3, 0]
[0, 6, 9, 10, 5, 11, 7, 2, 8, 4, 1, 12, 3, 0]
[0, 6, 9, 5, 11, 10, 7, 2, 8, 4, 1, 12, 3, 0]
[0, 6, 9, 5, 10, 7, 11, 2, 8, 4, 1, 12, 3, 0]
[0, 6, 9, 5, 10, 11, 2, 7, 8, 4, 1, 12, 3, 0]
[0, 6, 9, 5, 10, 11, 7, 8, 2, 4, 1, 12, 3, 0]
[0, 6, 9, 5, 10, 11, 7, 2, 4, 8, 1, 12, 3, 0]
[0, 6, 9, 5, 10, 11, 7, 2, 8, 1, 4, 12, 3, 0]
[0, 6, 9, 5, 10, 11, 7, 2, 8, 4, 12, 1, 3, 0]
[0, 6, 9, 5, 10, 11, 7, 2, 8, 4, 1, 3, 12, 0]


And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [30]:
#TODO: Apply hill climbing to TSP

initial_solution = TSPHillClimbingSolution()
hill_climbing_solution = hill_climbing(initial_solution, maximization = False, verbose = True)

print('Hill climbing solution:', hill_climbing_solution)

Current solution: [0, 9, 4, 6, 1, 8, 2, 11, 10, 5, 3, 12, 7, 0] with fitness 12768
Neighbor: [0, 4, 9, 6, 1, 8, 2, 11, 10, 5, 3, 12, 7, 0] with fitness 14227
Neighbor: [0, 9, 6, 4, 1, 8, 2, 11, 10, 5, 3, 12, 7, 0] with fitness 13568
Neighbor: [0, 9, 4, 1, 6, 8, 2, 11, 10, 5, 3, 12, 7, 0] with fitness 12853
Neighbor: [0, 9, 4, 6, 8, 1, 2, 11, 10, 5, 3, 12, 7, 0] with fitness 12374
Neighbor: [0, 9, 4, 6, 1, 2, 8, 11, 10, 5, 3, 12, 7, 0] with fitness 13310
Neighbor: [0, 9, 4, 6, 1, 8, 11, 2, 10, 5, 3, 12, 7, 0] with fitness 11486
Neighbor: [0, 9, 4, 6, 1, 8, 2, 10, 11, 5, 3, 12, 7, 0] with fitness 12917
Neighbor: [0, 9, 4, 6, 1, 8, 2, 11, 5, 10, 3, 12, 7, 0] with fitness 12832
Neighbor: [0, 9, 4, 6, 1, 8, 2, 11, 10, 3, 5, 12, 7, 0] with fitness 13611
Neighbor: [0, 9, 4, 6, 1, 8, 2, 11, 10, 5, 12, 3, 7, 0] with fitness 11929
Neighbor: [0, 9, 4, 6, 1, 8, 2, 11, 10, 5, 3, 7, 12, 0] with fitness 14663
Current solution: [0, 9, 4, 6, 1, 8, 11, 2, 10, 5, 3, 12, 7, 0] with fitness 11486
Neighbor:

The implementations of `TSPSolution` and `TSPHillClimbingSolution` can be found in `library/problems/tsp.py`

## Knapsack Problem

**Description:** The Knapsack Problem involves selecting a subset of N items, each with a given value and weight, to pack into a container with a fixed capacity. If the total weight of selected items exceeds the capacity, the solution is invalid. The goal is to maximize the total value of items while ensuring they fit within the container's constraints.

**Search space:** All possible subsets of items that can be placed in the knapsack.

**Representation:** Binary string of length N (number of items), where 1 indicates the item is included in the knapsack and 0 indicates the item is excluded.

**Fitness function:** f(x)= Total value inside the knapsack. If the total size exceeds the knapsack's capacity, the solution is invalid and assigned a fitness of -inf.

**Goal:** Maximize f(x).

Similarly to what we've done for TSP, let's begin by implementing the `KSSolution` class, which inherits from `Solution` and implementes the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Knapsack problem (KS) and does not include any implementation related to the optimization algorithm that will be used to solve it.

In [31]:
from library.problems.data.ks_data import values, weights, capacity

#TODO: Implement KSSolution class
class KSSolution(Solution):
    def __init__(self, repr=None, values = values, weights = weights, capacity = capacity):
        self.values = values
        self.weights = weights
        self.capacity = capacity
        super().__init__(repr = repr)
        
    def random_initial_representation(self):
        repr = []
        for i in range(len(self.values)):
            repr.append(choice([0, 1]))
            
        return repr
    
    def total_weight(self):
        total_weight = 0
        for i in range(len(self.repr)):
            if self.repr[i] == 1:
                total_weight += self.weights[i]
            
        return total_weight
    
    def total_value(self):
        total_value = 0
        for i in range(len(self.repr)):
            if self.repr[i] == 1:
                total_value += self.values[i]
                
        return total_value
    
    def fitness(self):
        total_weight = self.total_weight()
        
        if total_weight > self.capacity:
            return -9999999
        else:
            return self.total_value()

Let's test it

In [32]:
solution = KSSolution()

print(solution)
print(solution.fitness())

[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1]
-9999999


### Solving KS with Hill Climbing

A neighbor solution is obtained by flipping a single bit, meaning adding one item to the knapsack, or removing one item from the knapsack.

Let's create the `KSHillClimbingSolution` that inherits from `KSSolution` and implements the `get_neighbors` method.


In [35]:
#TODO: Implement KSHillClimbingSolution class
class KSHillClimbingSolution(KSSolution):
    def get_neighbors(self):
        neighbors = []
        
        for i in range(len(self.repr)):
            neighbor = deepcopy(self.repr)
            if self.repr[i] == 1:
                neighbor[i] = 0
            elif self.repr[i] == 0:
                neighbor[i] = 1
                
            neighbor = KSHillClimbingSolution(
                repr = neighbor, 
                values = self.values, 
                weights = self.weights, 
                capacity = self.capacity
            )
                
            neighbors.append(neighbor)
        return neighbors

Let's test it

In [36]:
solution = KSHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

Solution: [1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
Neihghbors:
[0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [38]:
#TODO: Apply hill climbing to KS problem
initial_solution = KSHillClimbingSolution()
hill_climbing(initial_solution, maximization = True, verbose = True, max_iter = 10)

Current solution: [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1] with fitness 4459
Neighbor: [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1] with fitness 4099
Neighbor: [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1] with fitness 4376
Neighbor: [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1] with fitness -9999999
Neighbor: [1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1] with fitness -9999999
Neighbor: [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 

[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]

The implementations of `KSSolution` and `KSHillClimbingSolution` can be found in `library/problems/knapsack.py`